I'm reading through several existing notebooks and trying to distill down the information into a new notebook to help me understand the project.  All help appreciated!

# References

- [Advanced EDA - Brain Tumor Data](https://www.kaggle.com/smoschou55/advanced-eda-brain-tumor-data)
- [Team 9 Second Week](https://www.kaggle.com/evanyao27/team-9-second-week)
  - The only model that is working. get_model02()
- [Dataset to Model with Tensorflow](https://www.kaggle.com/ohbewise/dataset-to-model-with-tensorflow)
- [Brain Tumer Train Class Flair](https://www.kaggle.com/lucamtb/brain-tumer-train-class-flair)
  - Uses TPU
  - Generates a Tensorflow model: Brain_flair_model_effect_3e-05_0.0001.h5
- [Brain Tumor very basic inference](https://www.kaggle.com/lucamtb/brain-tumor-very-basice-inference)
  - Uses the above mentioned model: Brain_flair_model_effect_3e-05_0.0001.h5
  - Add this Kaggle Dataset: https://www.kaggle.com/lucamtb/effect0-brain

# Load Libraries

In [1]:
import os
import glob

import pandas as pd
import numpy as np
from pathlib import Path

import random
from tqdm.notebook import tqdm
import pydicom # Handle MRI images

import cv2  # OpenCV - https://docs.opencv.org/master/d6/d00/tutorial_py_root.html

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers


2021-10-08 16:17:42.759949: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


# Configuration, Constants, Setup

In [2]:
data_dir = Path('../input/rsna-miccai-brain-tumor-radiogenomic-classification/')

mri_types = ["FLAIR", "T1w", "T2w", "T1wCE"]
excluded_images = [109, 123, 709] # Bad images

# Load Datasets

In [3]:
train_df = pd.read_csv(data_dir / "train_labels.csv",
#                        index='id',
#                       nrows=100000
                      )
test_df = pd.read_csv(data_dir / "sample_submission.csv")
sample_submission = pd.read_csv(data_dir / "sample_submission.csv")

train_df = train_df[~train_df.BraTS21ID.isin(excluded_images)]

print(f"train data: Rows={train_df.shape[0]}, Columns={train_df.shape[1]}")
# print(f"test data : Rows={test_df.shape[0]}, Columns={test_df.shape[1]}")

train data: Rows=582, Columns=2


# Utility Functions

### There's a version that converts into grayscale: 

- https://www.kaggle.com/smoschou55/advanced-eda-brain-tumor-data


In [4]:
def load_dicom(path, size = 224):
    ''' 
    Reads a DICOM image, standardizes so that the pixel values are between 0 and 1, then rescales to 0 and 255
    
    Not super sure if this kind of scaling is appropriate, but everyone seems to do it. 
    '''
    dicom = pydicom.read_file(path)
    data = dicom.pixel_array
    # transform data into black and white scale / grayscale
#     data = data - np.min(data)
    if np.max(data) != 0:
        data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return cv2.resize(data, (size, size))

In [5]:
def get_all_image_paths(brats21id, image_type, folder='train'): 
    '''
    Returns an arry of all the images of a particular type for a particular patient ID
    '''
    assert(image_type in mri_types)
    
    patient_path = os.path.join(
        "../input/rsna-miccai-brain-tumor-radiogenomic-classification/%s/" % folder, 
        str(brats21id).zfill(5),
    )

    paths = sorted(
        glob.glob(os.path.join(patient_path, image_type, "*")), 
        key=lambda x: int(x[:-4].split("-")[-1]),
    )
    
    num_images = len(paths)
    
    start = int(num_images * 0.25)
    end = int(num_images * 0.75)

    interval = 3
    
    if num_images < 10: 
        interval = 1
    
    return np.array(paths[start:end:interval])

def get_all_images(brats21id, image_type, folder='train', size=225):
    return [load_dicom(path, size) for path in get_all_image_paths(brats21id, image_type, folder)]

# Load Images We Will Need

In [6]:
def get_all_data_for_train(image_type, image_size=32):
    global train_df
    
    X = []
    y = []
    train_ids = []

    for i in tqdm(train_df.index):
        x = train_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'train', image_size)
        label = x['MGMT_value']

        X += images
        y += [label] * len(images)
        train_ids += [int(x['BraTS21ID'])] * len(images)
        assert(len(X) == len(y))
    return np.array(X), np.array(y), np.array(train_ids)

In [7]:
def get_all_data_for_test(image_type, image_size=32):
    global test_df
    
    X = []
    test_ids = []

    for i in tqdm(test_df.index):
        x = test_df.loc[i]
        images = get_all_images(int(x['BraTS21ID']), image_type, 'test', image_size)
        X += images
        test_ids += [int(x['BraTS21ID'])] * len(images)

    return np.array(X), np.array(test_ids)

In [8]:
X, y, trainidt = get_all_data_for_train('T1wCE', image_size=32)
X_test, testidt = get_all_data_for_test('T1wCE', image_size=32)

  0%|          | 0/582 [00:00<?, ?it/s]

  0%|          | 0/87 [00:00<?, ?it/s]

In [9]:
X.shape, y.shape, trainidt.shape

((16196, 32, 32), (16196,), (16196,))

# Train/Validation Split

In [10]:
X_train, X_valid, y_train, y_valid, trainidt_train, trainidt_valid = train_test_split(X, y, trainidt, test_size=0.2, random_state=42)

## Adding a Dimension

In [11]:
X_train.shape

(12956, 32, 32)

In [12]:
X_train = tf.expand_dims(X_train, axis=-1)
X_valid = tf.expand_dims(X_valid, axis=-1)

2021-10-08 16:20:10.836789: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-08 16:20:10.839951: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2021-10-08 16:20:10.876730: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-10-08 16:20:10.877381: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-10-08 16:20:10.877443: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-10-08 16:20:10.904230: I tensorflow/stream_executor/platform/def

In [13]:
X_train.shape

TensorShape([12956, 32, 32, 1])

## One-hot encode labels

In [14]:
y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

# Tensorflow Models

## Model 1
- from:  https://www.kaggle.com/ohbewise/dataset-to-model-with-tensorflow
- Keras code from here: https://keras.io/examples/vision/3D_image_classification/

## Define a 3D convolutional neural network

In [15]:
# Define, train, and evaluate model
# source: https://keras.io/examples/vision/3D_image_classification/
def get_model01(width=128, height=128, depth=64, name='3dcnn'):
    """Build a 3D convolutional neural network model."""

    inputs = tf.keras.Input((width, height, depth, 1))

    x = tf.keras.layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = tf.keras.layers.MaxPool3D(pool_size=2)(x)
    x = tf.keras.layers.BatchNormalization()(x)

    x = tf.keras.layers.GlobalAveragePooling3D()(x)
    x = tf.keras.layers.Dense(units=512, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    outputs = tf.keras.layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = tf.keras.Model(inputs, outputs, name=name)
    
    # Compile model.
    initial_learning_rate = 0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
    )
    model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
        metrics=["acc"],
    )
    
    return model



## Model 2

- from: https://www.kaggle.com/evanyao27/team-9-second-week/notebook
- Validation AUC=0.9148664856146349

In [16]:
def get_model02():
    np.random.seed(0)
    random.seed(12)
    tf.random.set_seed(12)

    inpt = keras.Input(shape=X_train.shape[1:])

    h = keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inpt)

    h = keras.layers.Conv2D(64, kernel_size=(4, 4), activation="relu", name="Conv_1")(h)
    h = keras.layers.MaxPool2D(pool_size=(2, 2))(h)

    h = keras.layers.Conv2D(32, kernel_size=(2, 2), activation="relu", name="Conv_2")(h)
    h = keras.layers.MaxPool2D(pool_size=(1, 1))(h)

    h = keras.layers.Dropout(0.1)(h)

    h = keras.layers.Flatten()(h)
    h = keras.layers.Dense(32, activation="relu")(h)

    output = keras.layers.Dense(2, activation="softmax")(h)

    model = keras.Model(inpt, output)

    roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')

    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=[roc_auc]
    )
    return model

## Model 3

Adding LR scheduler, early stopping, etc

In [17]:
def get_model03():
    np.random.seed(0)
    random.seed(12)
    tf.random.set_seed(12)

    inpt = keras.Input(shape=X_train.shape[1:])

    h = keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inpt)

    h = keras.layers.Conv2D(64, kernel_size=(4, 4), activation="relu", name="Conv_1")(h)
    h = keras.layers.MaxPool2D(pool_size=(2, 2))(h)

    h = keras.layers.Conv2D(32, kernel_size=(2, 2), activation="relu", name="Conv_2")(h)
    h = keras.layers.MaxPool2D(pool_size=(1, 1))(h)

    h = keras.layers.Dropout(0.1)(h)

    h = keras.layers.Flatten()(h)
    h = keras.layers.Dense(32, activation="relu")(h)

    output = keras.layers.Dense(2, activation="softmax")(h)

    model = keras.Model(inpt, output)

    # https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/schedules/ExponentialDecay
    
    initial_learning_rate =  0.0001
    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate,
        decay_steps=100000,
        decay_rate=0.96, 
        staircase=True
    )
  
    roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')

    model.compile(
        loss="categorical_crossentropy", 
#         loss="binary_crossentropy", 
        
#         optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        optimizer=keras.optimizers.Adam(),

        metrics=[roc_auc],
    )
    return model

## Model 4

In [18]:
def get_model04():
    np.random.seed(0)
    random.seed(12)
    tf.random.set_seed(12)

    inpt = keras.Input(shape=X_train.shape[1:])

    h = keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inpt)

    h = keras.layers.Conv2D(64, kernel_size=(20, 20), activation="relu", name="Conv_1")(h)
    h = keras.layers.MaxPool2D(pool_size=(8, 8))(h)

    #h = keras.layers.Conv2D(32, kernel_size=(2, 2), activation="relu", name="Conv_2")(h)
    #h = keras.layers.MaxPool2D(pool_size=(1, 1))(h)

    h = keras.layers.Dropout(0.1)(h)

    h = keras.layers.Flatten()(h)
    h = keras.layers.Dense(32, activation="relu")(h)

    output = keras.layers.Dense(2, activation="softmax")(h)

    model = keras.Model(inpt, output)

    roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')

    model.compile(
        loss="categorical_crossentropy", optimizer="adam", metrics=[roc_auc]
    )
    return model

## Model 5

## Set up Model Checkpoint

In [19]:
checkpoint_filepath = "best_model.h5"

model_checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor="val_roc_auc",
    mode="max",
    save_best_only=True,
    save_freq="epoch",
    verbose=1,
)

## Early Stopping Callback

- https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping

In [20]:
# early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor=tf.keras.metrics.AUC(), mode='auto', verbose=1, patience=5)
# early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)
early_stopping_cb = tf.keras.callbacks.EarlyStopping(monitor="val_roc_auc", mode='max', patience=3)

### Note that rerunning the cell below will change val_acc to val_acc_N and the model will not be saved.

Force name:  https://www.kaggle.com/c/rsna-miccai-brain-tumor-radiogenomic-classification/discussion/276230

    roc_auc = tf.keras.metrics.AUC(name='roc_auc', curve='ROC')
    model.compile(optimizer=..., loss=..., metrics=[roc_auc, ...])


In [21]:
model = get_model04() # LB score 0.676
#model = get_model03() # LB score 0.5
model.summary()

#enable voting of the 3 models:


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 1)]       0         
_________________________________________________________________
rescaling (Rescaling)        (None, 32, 32, 1)         0         
_________________________________________________________________
Conv_1 (Conv2D)              (None, 13, 13, 64)        25664     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 1, 1, 64)          0         
_________________________________________________________________
dropout (Dropout)            (None, 1, 1, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                2080  

In [22]:
# history = model.fit(x=X_train, y = y_train, epochs=20, 
#                     callbacks=[model_checkpoint_cb], 
#                     validation_data=(X_valid, y_valid))

history = model.fit(x=X_train, y = y_train, epochs=40, 
                    callbacks=[model_checkpoint_cb, early_stopping_cb],
                    validation_data=(X_valid, y_valid))

2021-10-08 16:20:14.496538: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-08 16:20:14.507534: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2000160000 Hz


Epoch 1/40


2021-10-08 16:20:15.142254: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2021-10-08 16:20:15.982342: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2021-10-08 16:20:16.006680: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.8


405/405 [==============================] - 9s 5ms/step - loss: 0.6883 - roc_auc: 0.5673 - val_loss: 0.6813 - val_roc_auc: 0.5917

Epoch 00001: val_roc_auc improved from -inf to 0.59173, saving model to best_model.h5
Epoch 2/40
405/405 [==============================] - 2s 4ms/step - loss: 0.6817 - roc_auc: 0.5906 - val_loss: 0.6829 - val_roc_auc: 0.5855

Epoch 00002: val_roc_auc did not improve from 0.59173
Epoch 3/40
405/405 [==============================] - 2s 4ms/step - loss: 0.6791 - roc_auc: 0.5961 - val_loss: 0.6703 - val_roc_auc: 0.6193

Epoch 00003: val_roc_auc improved from 0.59173 to 0.61926, saving model to best_model.h5
Epoch 4/40
405/405 [==============================] - 2s 4ms/step - loss: 0.6705 - roc_auc: 0.6181 - val_loss: 0.6677 - val_roc_auc: 0.6215

Epoch 00004: val_roc_auc improved from 0.61926 to 0.62150, saving model to best_model.h5
Epoch 5/40
405/405 [==============================] - 2s 4ms/step - loss: 0.6710 - roc_auc: 0.6178 - val_loss: 0.6646 - val_roc_a

# Load Our Best Model

In [23]:
model_best = tf.keras.models.load_model(filepath=checkpoint_filepath)

# Predictions on Validation Set

In [24]:
y_pred = model_best.predict(X_valid)

pred = np.argmax(y_pred, axis=1)

result = pd.DataFrame(trainidt_valid)
result[1] = pred

result.columns = ["BraTS21ID", "MGMT_value"]
result2 = result.groupby("BraTS21ID", as_index=False).mean()

result2 = result2.merge(train_df, on="BraTS21ID")
auc = roc_auc_score(
    result2.MGMT_value_y,
    result2.MGMT_value_x,
)
print(f"Validation AUC={auc}")

Validation AUC=0.759942927252926


# Predictions on the Test Set

In [25]:
y_pred = model_best.predict(X_test)

pred = np.argmax(y_pred, axis=1) #

result = pd.DataFrame(testidt)
result[1] = pred
pred

array([1, 1, 0, ..., 1, 1, 1])

# Submission File

In [26]:
result.columns=['BraTS21ID','MGMT_value']

result2 = result.groupby('BraTS21ID',as_index=False).mean()
result2['BraTS21ID'] = sample_submission['BraTS21ID']

# Rounding... 0.907866 -> 0.9
result2['MGMT_value'] = result2['MGMT_value'].apply(lambda x:round(x*10)/10)
# result2['MGMT_value'] = result2['MGMT_value'] # No rounding
result2.to_csv('submission.csv',index=False)
result2

,BraTS21ID,MGMT_value
0,1,0.9
1,13,0.7
2,15,1.0
3,27,1.0
4,37,1.0
...,...,...
82,826,0.4
83,829,0.2
84,833,0.3
85,997,0.3
